In [1]:
import tweepy
import json
import pandas as pd
import random
from dotenv import load_dotenv
import os
import time
from datetime import datetime
load_dotenv()

True

# Sentiment analysis de tweets respecto al plebiscito en Septiembre 2022 Chile

Este script busca en twiter, con la los privilegios de la cuenta research, los tweets según las palabras clave, la localización y las fechas determinadas.

La información mas importante de cada tweet es guardado en un dataframe.

La API libre de twitter tiene varias limitaciones, entre ella la fecha:

    Keep in mind that the search index has a 7-day limit. In other words, no tweets will be found for a date older than one week.
    
No se puede traer tweets mas antiguos que una semana. Ademas del peso sobre la cantidad de tweets, que cuelga la API por 15 minutos

##### Definición de palabras clave (_keywords_) a buscar dentro del texto de los tweets:

In [2]:
keywords_list = ['Constituyente',
'Convención Constituyente',
'Convención Constitucional',
'Asamblea Constituyente',
'plebiscito',
'referéndum',
'constitución chilena',
'Acuerdo por la paz social y la Nueva Constitución',
'Vamos por Chile',
'Unión Demócrata Independiente',
'Constituyentes Apartidistas',
'Independientes No Neutrales',
'Lista del Pueblo',
'Apruebo Dignidad',
'Lista del Apruebo',
'Pueblo Constituyente']

keywords_list_II = ['Chile Digno',
'Chile Unido',
'Unidos por Chile'
'Chile Libre',
'Coordinadora Constituyente Plurinacional y Popular',
'Somos región',
'Colectivo Socialista',
'Independientes por una Nueva Constitución',
'Movimientos sociales constituyentes',
'Colectivo del Apruebo',
'Wallmapur',
'Elisa Loncón',
'Campaña de desprestigio',
'rechazo del prebiscito de 2020',
'Carmen Gloria Valladares Moyano',
'Elisa Loncon',
'Destitucion de Elisa Loncon']

In [3]:
keywords_list = ["(" + word + ")" for word in keywords_list]

'#DestitucionDeElisaLoncon',
'juicio político',
'Jaime Bassa',
'Marcela Cubillos',
'Jaime Mañalich',
'Autodesprestigio',
'María Elisa Quinteros Cáceres',
'Gaspar Dominguez Donoso']

In [4]:
keywords_list  = ' OR '.join(keywords_list)

In [5]:
keywords_list

'(Constituyente) OR (Convención Constituyente) OR (Convención Constitucional) OR (Asamblea Constituyente) OR (plebiscito) OR (referéndum) OR (constitución chilena) OR (Acuerdo por la paz social y la Nueva Constitución) OR (Vamos por Chile) OR (Unión Demócrata Independiente) OR (Constituyentes Apartidistas) OR (Independientes No Neutrales) OR (Lista del Pueblo) OR (Apruebo Dignidad) OR (Lista del Apruebo) OR (Pueblo Constituyente)'

#### Funciones
El script tiene dos funciones principales:
- Activación de la API con las credenciales
- Búsqueda de los tweets con lo parámetros definidos

##### Función Activación

In [6]:
# Función de generación de la API
def oauth():
    #Pass in our twitter API authentication key
    consumer_key = os.getenv('CONSUMER_KEY')
    consumer_secret = os.getenv('CONSUMER_SECRET')
    access_token = os.getenv('ACCESS_TOKEN')
    access_token_secret = os.getenv('ACCESS_TOKEN_SECRET')
    auth = tweepy.OAuth1UserHandler(
        consumer_key, consumer_secret,
        access_token, access_token_secret
    )
#     Instantiate the tweepy API
    api = tweepy.API(auth, wait_on_rate_limit=True)
    return api

##### Función Búsqueda

In [28]:
# Función de búsqueda de tweets según los keywords, cantidad y localización
def search(geo, search_query, page_limit):
    page_no = 1
    # Creamos las columnas para el DataFrame
    data = []
    dict = {}
    page_no = 1

    df = pd.DataFrame()
    columns = ['Time','User', 'Tweet']
    try:
        # Lanzamos la búsqueda
        pages = tweepy.Cursor(api.search_tweets,
                              q=search_query, 
                              tweet_mode='extended', 
                              geocode = geo).pages(page_limit)
        print(pages)
    # Iterate through the pages and print the text of the tweets
        for page in pages:    
            for i in range(len(page)):
                data.append([page[i].created_at, page[i].user.screen_name, page[i].full_text])
            page_no += 1
        df = pd.DataFrame(data, columns=columns)
        return df
    
    except BaseException as e:
        print('Status Failed On,',str(e))
        return

##### Función Guardar

In [36]:
def save_func(df, file_type):
    filename = 'plebiscito_' + datetime.now().strftime('%m_%d_%Y_%H:%M:%S')
    if file_type == 'xlsx':
        df['Time'] = df['Time'].dt.tz_localize(None) # Excel no soporta Datatime con zonas
        filename = filename + '.xlsx'
        df.to_excel(filename, index = False)
    if file_type == 'csv':
        df.replace(',',' ', regex=True, inplace=True)
        df.replace('\n','', regex=True, inplace=True)
        filename = filename + '.csv'
        df.to_csv(filename, index = False)
        

#### Ejecución 

##### Instanciamos la API

In [29]:
api = oauth()
api

#####  Cargamos los parámetros de búsqueda


- `sgo`: latitud y longitud de Santiago de Chile
- `radio`: radio en km alrededor del punto
- `no_of_tweets`: Cantidad de tweets para cada keyword
- `geo`: arma el feature según lo cargado anteriormente, con la ciudad y el radio.
- `keywords_list`: Se incluye en esta sección sólo para testear. Se limita a dos palabras. Si se comenta esa línea, usa toda la lista presentada arriba)

In [30]:
sgo = '-33.4489,-70.6693'
radio = '100km'
page_limit =2
geo = str(sgo) + ',' + str(radio) 
# keywords_list = ['Chile'] # Comentar esta linea para usar toda la keywords_list


In [31]:
keywords_list

'(Constituyente) OR (Convención Constituyente) OR (Convención Constitucional) OR (Asamblea Constituyente) OR (plebiscito) OR (referéndum) OR (constitución chilena) OR (Acuerdo por la paz social y la Nueva Constitución) OR (Vamos por Chile) OR (Unión Demócrata Independiente) OR (Constituyentes Apartidistas) OR (Independientes No Neutrales) OR (Lista del Pueblo) OR (Apruebo Dignidad) OR (Lista del Apruebo) OR (Pueblo Constituyente)'

#####  Lanzamos la búsqueda

In [32]:
# df = pd.DataFrame()
df = search(geo,
            keywords_list,
            page_limit)


In [33]:
pd.set_option('display.max_colwidth', None)
df

,Time,User,Tweet
0,2022-08-24 18:23:41+00:00,juankvergarac,"@CalderonBassi Quiero pensar bien, pero creo que es un show antes del plebiscito y luego del 4 lo van a soltar. Espero equivocarme 🫣"
1,2022-08-24 18:23:27+00:00,Mazinger_Robot,"La detención por parte del desgobierno de Boric al terrorista Héctor Llaitul no es más que un show, un mero acto de campaña: después del plebiscito, lo sueltan... \nPor eso y mucho más, voto #Rechazo."
2,2022-08-24 18:23:27+00:00,kattmiel,@T13 Este gobierno jura q la gente es weona . Este narcoterrorista después del plebiscito estará libre. Tenemos el peor y más corrupto @PJudicialChile y @FiscaliadeChile #BoricNoEsMiPresidente #NoMasPicoEnElOjo #ServelCorrupto #RechazoPopular https://t.co/iqDlwsoe9H
3,2022-08-24 18:23:10+00:00,Mazinger_Robot,"La detención por parte del desgobierno de Boric al terrorista Héctor Llaitul no es más que un mero acto de campaña: después del plebiscito, lo sueltan... \nPor eso y mucho más, voto #Rechazo."
4,2022-08-24 18:22:49+00:00,Mazinger_Robot,"La detención por parte del desgobierno de Boric al terrorista Héctor Llaitul no es más que un mero acto de campaña: después del plebiscito, lo sueltan..."
5,2022-08-24 18:22:06+00:00,Cristobal__Hdz,"Bien ahí, aunque sea netamente por ganar el plebiscito. A este ritmo para conquistar votos van a firmar el tpp 11, recuperar la Patagonia y por ahí capaz que anuncien un estadio para la U https://t.co/VyruLkIBcb"
6,2022-08-24 18:22:00+00:00,latercera,"🎙 Podcast - El Café Diario | El día después: ¿qué pasará el 5 de septiembre? \nA 11 días del plebiscito, los distintos sectores políticos alistan sus cartas para el día después https://t.co/W5PUsqR3vo https://t.co/OzZTrIdgOo"
7,2022-08-24 18:21:58+00:00,vonjuanpablo,@CaritOROIII @danieljadue show time pre plebiscito
8,2022-08-24 18:21:54+00:00,felipe_h_1980,@biobio Raro a pocos días del plebiscito
9,2022-08-24 18:21:31+00:00,Dacauuu,"@Cooperativa El Soros le metió grafeno al Llaitul para dejarse arrestar, es un plan orquestado por la agrupación transplanetaria narcochavista para dejar bien parado al Boris antes del plebiscito. Me cuenta David, taxista que vota rechazo,mientras se limpia un polvo blanco del bigote."


#####  Guardamos el dF

In [39]:
save_func(df, 'csv')

In [40]:
save_func(df, 'xlsx')